In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import lightning as L
import pytorch_lightning as pl
from fastdtw import fastdtw

import matplotlib.pyplot as plt
plt.rcParams.update({
    'font.family':'Times New Roman', 
    'font.size': 14, 
    'axes.titlesize': 16, 
    'axes.labelsize': 14, 
    'xtick.labelsize': 12, 
    'ytick.labelsize': 12,
    'legend.fontsize': 14, 
})

# 1. Load Data

In [ ]:
import pickle

def load_from_file(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

train_A = load_from_file('dataset/train_A.pkl')
valid_A = load_from_file('dataset/valid_A.pkl')
test_A = load_from_file('dataset/test_A.pkl')

train_B = load_from_file('dataset/train_B.pkl')
valid_B = load_from_file('dataset/valid_B.pkl')
test_B = load_from_file('dataset/test_B.pkl')

# 2.DataLoader

In [ ]:
def generate_subsequences(data_list, sub_length=288, ratio=0.75):

    all_subsequences = []

    for time_series in data_list:
        step = int(sub_length * (1 - ratio))
        num_subsequences = (len(time_series) - sub_length) // step + 1

        for i in range(int(num_subsequences)):
            start_index = int(i * step)
            end_index = start_index + sub_length
            
            if end_index > len(time_series):
                break
            
            subsequence = time_series[start_index:end_index]
            all_subsequences.append(subsequence)
    return np.array(all_subsequences)

In [ ]:
def get_loader(data, batch_size, sub_length=288, ratio=0.5, shuffle=True):
    array = generate_subsequences(data, sub_length, ratio)

    tensor = torch.tensor(array, dtype=torch.float32).unsqueeze(-1)

    dataset = TensorDataset(tensor)

    dataloader = DataLoader(dataset, 
                            batch_size=batch_size, 
                            shuffle=shuffle, 
                            # num_workers=2  
                            )
    return dataloader

In [ ]:
batch_size = 16
sub_length = 288
ratio = 0.75

train_A_loader = get_loader(train_A, batch_size, sub_length, ratio)

In [ ]:
for data in train_A_loader:
    print(data[0].shape)
    break

# 3.Models

### Incoder

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim):
        super(Encoder, self).__init__()
        self.input_dim = input_dim
        self.embedding_dim = embedding_dim
        self.hidden_dim = 2 * embedding_dim

        self.gru1 = nn.GRU(
            input_size=self.input_dim,
            hidden_size=self.hidden_dim,
            num_layers=1,
            batch_first=True,  
        )

        self.gru2 = nn.GRU(
            input_size=self.hidden_dim,
            hidden_size=embedding_dim,
            num_layers=1,
            batch_first=True,
        )

    def forward(self, x):
        batch_size = x.shape[0]  
        x, h_n = self.gru1(x)
        x, h_n = self.gru2(x)
        return h_n

### Decoder

In [ ]:
class Decoder_2(nn.Module):
    def __init__(self, seq_length, input_dim, embedding_dim):
        super(Decoder_2, self).__init__()
        self.seq_length = seq_length
        self.input_dim = input_dim

        self.embedding_dim = embedding_dim
        self.hidden_dim  = 2 * embedding_dim
        
        self.gru1 = nn.GRU(
            input_size=input_dim,
            hidden_size=embedding_dim,
            num_layers=1,
            batch_first=True
        )

        self.gru2 = nn.GRU(
            input_size=embedding_dim,
            hidden_size=self.hidden_dim,
            num_layers=1,
            batch_first=True,
        )

        self.output_layer = nn.Linear(self.hidden_dim,
                                      self.input_dim)

    def forward(self, xh, targets=None, teacher_forcing_ratio=0):
        batch_size = xh.size(1)

        if targets is not None:
            decoder_input = targets[:, 0:1, :] 
        else:
            decoder_input = torch.zeros(batch_size, 1, self.input_dim, device=xh.device)

        decoder_hidden1 = xh  
        decoder_hidden2 = torch.zeros(1, batch_size, self.hidden_dim, device=xh.device)  

        outputs = []
        seq_len = self.seq_length
        for t in range(seq_len):
            out, decoder_hidden1 = self.gru1(decoder_input, decoder_hidden1)
            out, decoder_hidden2  = self.gru2(out, decoder_hidden2)

            out = self.output_layer(out)
            outputs.append(out)

            if targets is not None and torch.rand(1).item() < teacher_forcing_ratio:
                decoder_input = targets[:, t:t+1, :]
            else:
                # decoder_input = out
                decoder_input = out.detach()  

        outputs = torch.cat(outputs, dim=1)
        return outputs

### Loss Function

In [ ]:
# def dtw_loss(y_true, y_pred):
#     y_true_np = y_true.detach().cpu().numpy()
#     y_pred_np = y_pred.detach().cpu().numpy()
#     total_dtw_distance = 0
    
#     for true, pred in zip(y_true_np, y_pred_np):
#         distance, _ = fastdtw(true, pred) 
#         total_dtw_distance += distance
    
#     return torch.tensor(total_dtw_distance / y_true.size(0), requires_grad=True)

# class CombinedDTW_MSE_Loss(nn.Module):
#     def __init__(self, alpha=0.5):
#         super(CombinedDTW_MSE_Loss, self).__init__()
#         self.alpha = alpha  
#         self.mse_loss = nn.MSELoss()  

#     def forward(self, y_true, y_pred):
#         mse = self.mse_loss(y_true, y_pred)
#         dtw = dtw_loss(y_true, y_pred)
#         combined_loss = self.alpha * mse + (1 - self.alpha) * dtw
#         return combined_loss

In [ ]:
def fluctuation_sensitive_loss(y_true, y_pred):
    delta_true = y_true[:, 1:] - y_true[:, :-1]  
    delta_pred = y_pred[:, 1:] - y_pred[:, :-1]  
    loss = torch.mean((delta_true - delta_pred) ** 2).to(y_true.device)
    return loss

class CombinedLoss(nn.Module):
    def __init__(self, alpha=0.5):
        super(CombinedLoss, self).__init__()
        self.alpha = alpha
        self.mse_loss = nn.MSELoss()

    def forward(self, y_true, y_pred):
        mse = self.mse_loss(y_true, y_pred)
        fluctuation_loss = fluctuation_sensitive_loss(y_true, y_pred)
        combined_loss = self.alpha * mse + (1 - self.alpha) * fluctuation_loss
        return combined_loss

### Autoencoder

In [ ]:
class AutoencoderGRU_2(pl.LightningModule):
    def __init__(self, seq_length, input_dim, embedding_dim, teacher_forcing_ratio=0, alpha=0.5):
        super().__init__()
        self.encoder = Encoder(input_dim, embedding_dim)
        self.decoder = Decoder_2(seq_length, input_dim, embedding_dim)
        self.teacher_forcing_ratio = teacher_forcing_ratio

        self.training_step_outputs = []
        self.validation_step_outputs = []

        self.train_losses = []
        self.val_losses = []

        self.loss_fn = CombinedLoss(alpha=alpha)
    
    def forward(self, x, teacher_forcing_ratio=None):
        if teacher_forcing_ratio is None:
            teacher_forcing_ratio = self.teacher_forcing_ratio
        xh = self.encoder(x)
        out = self.decoder(xh, targets=x, teacher_forcing_ratio=teacher_forcing_ratio)
        return out
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
        return optimizer

    def training_step(self, batch, batch_idx):
        x = batch[0]
        pred = self(x)
        loss = self.loss_fn(pred, x)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x = batch[0]
        pred = self(x, teacher_forcing_ratio=0.0)
        loss = self.loss_fn(pred, x)
        self.log("val_loss", loss)
        return loss
        
    def test_step(self, batch, batch_idx):
        x = batch[0]
        pred = self(x, teacher_forcing_ratio=0.0)
        loss = self.loss_fn(pred, x)
        self.log("test_loss", loss)
        return loss

    def predict_step(self, batch, batch_idx):
        x = batch[0]
        pred = self(x, teacher_forcing_ratio=0.0)
        return pred
    
    def on_train_epoch_end(self):
        train_loss = self.trainer.callback_metrics.get('train_loss')
        if train_loss is not None:
            self.train_losses.append(train_loss.cpu().detach().item())

    def on_validation_epoch_end(self):
        val_loss = self.trainer.callback_metrics.get('val_loss')
        if val_loss is not None:
            self.val_losses.append(val_loss.cpu().detach().item())

# 4. Trainer

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from matplotlib.ticker import MaxNLocator
# from pytorch_lightning.loggers import WandbLogger
# from pytorch_lightning.loggers import TensorBoardLogger

class PlotLossesCallback(pl.Callback):
    def __init__(self, name="ex_id"):
        self.name = name
    def on_train_end(self, trainer, pl_module):
        plt.figure(figsize=(8, 5)) 
        plt.plot(pl_module.train_losses, label='Training Loss', linestyle='-', color='b')
        if len(pl_module.val_losses) > 1: 
            plt.plot(pl_module.val_losses[1:], label='Validation Loss', linestyle='--', color='r')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')

        plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))

        plt.legend()
        plt.title('Training and Validation Loss During Model Training')
        plt.grid(True)  
        plt.savefig(f'{self.name} loss.svg', dpi=300) 
        plt.show()

def get_trainer(name="ex_id"):
    # tf_logger = TensorBoardLogger("tb_logs", name=name)
    # wandb_logger = WandbLogger(project=name)

    checkpoint_callback = ModelCheckpoint(
        monitor='val_loss',             
        dirpath=f'checkpoint/{name}',  
        filename='{epoch:02d}-{val_loss:.2f}', 
        save_top_k=3, 
        mode='min'
    )

    early_stopping_callback = EarlyStopping(
        monitor='val_loss',            
        patience=5,       
        mode='min',  
        verbose=True  
    )
    
    trainer = pl.Trainer(
        max_epochs=100,
        log_every_n_steps=1,
        callbacks=[
            checkpoint_callback, 
            # early_stopping_callback, 
            PlotLossesCallback(name=name)
            ],
        # logger = wandb_logger
    )
    return trainer

# 5. Experiment

In [ ]:
train_A_loader_288 = get_loader(data=train_A, batch_size=32, sub_length=288, ratio=0.75)
train_B_loader_288 = get_loader(data=train_B, batch_size=32, sub_length=288, ratio=0.75)

val_A_loader_288 = get_loader(data=valid_A, batch_size=32, sub_length=288, ratio=0.75, shuffle=False)
val_B_loader_288 = get_loader(data=valid_B, batch_size=32, sub_length=288, ratio=0.75, shuffle=False)

### 1-1

In [ ]:
seq_length=288
embedding_dim = int(seq_length / 2) 
teacher_forcing_ratio = 0 

model1_1 = AutoencoderGRU_2(
    seq_length=seq_length,
    input_dim=1,
    embedding_dim=embedding_dim,
    teacher_forcing_ratio=teacher_forcing_ratio
)

trainer1_1 = get_trainer("ex1_1")
trainer1_1.fit(model1_1, train_A_loader_288, val_A_loader_288)

### 1-2

In [ ]:
seq_length=288
embedding_dim = int(seq_length / 2)
teacher_forcing_ratio = 0.25

model1_2 = AutoencoderGRU_2(
    seq_length=seq_length,
    input_dim=1,
    embedding_dim=embedding_dim,
    teacher_forcing_ratio=teacher_forcing_ratio
)

trainer1_2 = get_trainer("ex1_2")
trainer1_2.fit(model1_2, train_A_loader_288, val_A_loader_288)

### 1-3

In [ ]:
seq_length=288
embedding_dim = int(seq_length / 2)
teacher_forcing_ratio = 0.5 

model1_3 = AutoencoderGRU_2(
    seq_length=seq_length,
    input_dim=1,
    embedding_dim=embedding_dim,
    teacher_forcing_ratio=teacher_forcing_ratio
)

trainer1_3 = get_trainer("ex1_3")
trainer1_3.fit(model1_3, train_A_loader_288, val_A_loader_288)

### 1-4

In [ ]:
seq_length=288
embedding_dim = int(seq_length / 2) 
teacher_forcing_ratio = 0.75 

model1_4 = AutoencoderGRU_2(
    seq_length=seq_length,
    input_dim=1,
    embedding_dim=embedding_dim,
    teacher_forcing_ratio=teacher_forcing_ratio
)

trainer1_4 = get_trainer("ex1_4")
trainer1_4.fit(model1_4, train_A_loader_288, val_A_loader_288)

### 1-5

In [ ]:
seq_length=288
embedding_dim = int(seq_length / 2)
teacher_forcing_ratio = 1

model1_5 = AutoencoderGRU_2(
    seq_length=seq_length,
    input_dim=1,
    embedding_dim=embedding_dim,
    teacher_forcing_ratio=teacher_forcing_ratio
)

trainer1_5 = get_trainer("ex1_5")
trainer1_5.fit(model1_5, train_A_loader_288, val_A_loader_288)

### 1-6

In [ ]:
seq_length=288
embedding_dim = int(seq_length / 2)
teacher_forcing_ratio = 0 

model1_6 = AutoencoderGRU_2(
    seq_length=seq_length,
    input_dim=1,
    embedding_dim=embedding_dim,
    teacher_forcing_ratio=teacher_forcing_ratio
)

trainer1_6 = get_trainer("ex1_6")
trainer1_6.fit(model1_6, train_B_loader_288, val_B_loader_288)

### 1-7

In [ ]:
seq_length=288
embedding_dim = int(seq_length / 2) 
teacher_forcing_ratio = 0.25 

model1_7 = AutoencoderGRU_2(
    seq_length=seq_length,
    input_dim=1,
    embedding_dim=embedding_dim,
    teacher_forcing_ratio=teacher_forcing_ratio
)

trainer1_7 = get_trainer("ex1_7")
trainer1_7.fit(model1_7, train_B_loader_288, val_B_loader_288)

### 1-8

In [ ]:
seq_length=288
embedding_dim = int(seq_length / 2) 
teacher_forcing_ratio = 0.5 

model1_8 = AutoencoderGRU_2(
    seq_length=seq_length,
    input_dim=1,
    embedding_dim=embedding_dim,
    teacher_forcing_ratio=teacher_forcing_ratio
)

trainer1_8 = get_trainer("ex1_8")
trainer1_8.fit(model1_8, train_B_loader_288, val_B_loader_288)

### 1-9

In [ ]:
seq_length=288
embedding_dim = int(seq_length / 2)
teacher_forcing_ratio = 0.75 

model1_9 = AutoencoderGRU_2(
    seq_length=seq_length,
    input_dim=1,
    embedding_dim=embedding_dim,
    teacher_forcing_ratio=teacher_forcing_ratio
)

trainer1_9 = get_trainer("ex1_9")
trainer1_9.fit(model1_9, train_B_loader_288, val_B_loader_288)

### 1-10

In [ ]:
seq_length=288
embedding_dim = int(seq_length / 2)
teacher_forcing_ratio = 0.9

model1_10 = AutoencoderGRU_2(
    seq_length=seq_length,
    input_dim=1,
    embedding_dim=embedding_dim,
    teacher_forcing_ratio=teacher_forcing_ratio
)

trainer1_10 = get_trainer("ex1_10")
trainer1_10.fit(model1_10, train_B_loader_288, val_B_loader_288)

### dump loss

In [ ]:
experiment_1_loss = [model1_1.train_losses, model1_1.val_losses,
                     model1_2.train_losses, model1_2.val_losses,
                     model1_3.train_losses, model1_3.val_losses,
                     model1_4.train_losses, model1_4.val_losses,
                     model1_5.train_losses, model1_5.val_losses,
                     model1_6.train_losses, model1_6.val_losses,
                     model1_7.train_losses, model1_7.val_losses,
                     model1_8.train_losses, model1_8.val_losses,
                     model1_9.train_losses, model1_9.val_losses,
                     model1_10.train_losses, model1_10.val_losses]

with open('experiment_1_loss.pkl', 'wb') as f:
    pickle.dump(experiment_1_loss, f)

In [ ]:
train_A_loader_288 = get_loader(data=train_A, batch_size=32, sub_length=288, ratio=0.75)
train_B_loader_288 = get_loader(data=train_B, batch_size=32, sub_length=288, ratio=0.75)

val_A_loader_288 = get_loader(data=valid_A, batch_size=32, sub_length=288, ratio=0.75, shuffle=False)
val_B_loader_288 = get_loader(data=valid_B, batch_size=32, sub_length=288, ratio=0.75, shuffle=False)

train_A_loader_144 = get_loader(data=train_A, batch_size=32, sub_length=144, ratio=0.75)
train_B_loader_144 = get_loader(data=train_B, batch_size=32, sub_length=144, ratio=0.75)

val_A_loader_144 = get_loader(data=valid_A, batch_size=32, sub_length=144, ratio=0.75, shuffle=False)
val_B_loader_144 = get_loader(data=valid_B, batch_size=32, sub_length=144, ratio=0.75, shuffle=False)

train_A_loader_216 = get_loader(data=train_A, batch_size=32, sub_length=216, ratio=0.75)
train_B_loader_216 = get_loader(data=train_B, batch_size=32, sub_length=216, ratio=0.75)

val_A_loader_216 = get_loader(data=valid_A, batch_size=32, sub_length=216, ratio=0.75, shuffle=False)
val_B_loader_216 = get_loader(data=valid_B, batch_size=32, sub_length=216, ratio=0.75, shuffle=False)

train_A_loader_360 = get_loader(data=train_A, batch_size=32, sub_length=360, ratio=0.75)
train_B_loader_360 = get_loader(data=train_B, batch_size=32, sub_length=360, ratio=0.75)

val_A_loader_360 = get_loader(data=valid_A, batch_size=32, sub_length=360, ratio=0.75, shuffle=False)
val_B_loader_360 = get_loader(data=valid_B, batch_size=32, sub_length=360, ratio=0.75, shuffle=False)

train_A_loader_432 = get_loader(data=train_A, batch_size=32, sub_length=432, ratio=0.75)
train_B_loader_432 = get_loader(data=train_B, batch_size=32, sub_length=432, ratio=0.75)

val_A_loader_432 = get_loader(data=valid_A, batch_size=32, sub_length=432, ratio=0.75, shuffle=False)
val_B_loader_432 = get_loader(data=valid_B, batch_size=32, sub_length=432, ratio=0.75, shuffle=False)


### 2-1

In [ ]:
seq_length=144
embedding_dim = int(seq_length / 2)
teacher_forcing_ratio = 0 

model2_1 = AutoencoderGRU_2(
    seq_length=seq_length,
    input_dim=1,
    embedding_dim=embedding_dim,
    teacher_forcing_ratio=teacher_forcing_ratio,
    alpha=0.5
)

trainer2_1 = get_trainer("ex2_1")
trainer2_1.fit(model2_1, train_A_loader_144, val_A_loader_144)

### 2-2

In [ ]:
seq_length = 216
embedding_dim = int(seq_length / 2) 
teacher_forcing_ratio = 0

model2_2 = AutoencoderGRU_2(
    seq_length=seq_length,
    input_dim=1,
    embedding_dim=embedding_dim,
    teacher_forcing_ratio=teacher_forcing_ratio
)

trainer2_2 = get_trainer("ex2_2")
trainer2_2.fit(model2_2, train_A_loader_216, val_A_loader_216)

### 2-3

In [ ]:
seq_length = 288
embedding_dim = int(seq_length / 2) 
teacher_forcing_ratio = 0 

model2_3 = AutoencoderGRU_2(
    seq_length=seq_length,
    input_dim=1,
    embedding_dim=embedding_dim,
    teacher_forcing_ratio=teacher_forcing_ratio
)

trainer2_3 = get_trainer("ex2_3")
trainer2_3.fit(model2_3, train_A_loader_288, val_A_loader_288)


### 2-4

In [ ]:
seq_length = 360
embedding_dim = int(seq_length / 2) 
teacher_forcing_ratio = 0 

model2_4 = AutoencoderGRU_2(
    seq_length=seq_length,
    input_dim=1,
    embedding_dim=embedding_dim,
    teacher_forcing_ratio=teacher_forcing_ratio
)

trainer2_4 = get_trainer("ex2_4")
trainer2_4.fit(model2_4, train_A_loader_360, val_A_loader_360)


### 2-5

In [ ]:
seq_length = 432
embedding_dim = int(seq_length / 2) 
teacher_forcing_ratio = 0 

model2_5 = AutoencoderGRU_2(
    seq_length=seq_length,
    input_dim=1,
    embedding_dim=embedding_dim,
    teacher_forcing_ratio=teacher_forcing_ratio
)

trainer2_5 = get_trainer("ex2_5")
trainer2_5.fit(model2_5, train_A_loader_432, val_A_loader_432)


### 2-6

In [ ]:
seq_length = 144
embedding_dim = int(seq_length / 2)
teacher_forcing_ratio = 0

model2_6 = AutoencoderGRU_2(
    seq_length=seq_length,
    input_dim=1,
    embedding_dim=embedding_dim,
    teacher_forcing_ratio=teacher_forcing_ratio
)

trainer2_6 = get_trainer("ex2_6")
trainer2_6.fit(model2_6, train_B_loader_144, val_B_loader_144)


### 2-7

In [ ]:
seq_length = 216
embedding_dim = int(seq_length / 2)
teacher_forcing_ratio = 0

model2_7 = AutoencoderGRU_2(
    seq_length=seq_length,
    input_dim=1,
    embedding_dim=embedding_dim,
    teacher_forcing_ratio=teacher_forcing_ratio
)

trainer2_7 = get_trainer("ex2_7")
trainer2_7.fit(model2_7, train_B_loader_216, val_B_loader_216)


### 2-8

In [ ]:
seq_length = 288
embedding_dim = int(seq_length / 2)
teacher_forcing_ratio = 0 

model2_8 = AutoencoderGRU_2(
    seq_length=seq_length,
    input_dim=1,
    embedding_dim=embedding_dim,
    teacher_forcing_ratio=teacher_forcing_ratio
)

trainer2_8 = get_trainer("ex2_8")
trainer2_8.fit(model2_8, train_B_loader_288, val_B_loader_288)


### 2-9

In [ ]:
seq_length = 360
embedding_dim = int(seq_length / 2)
teacher_forcing_ratio = 0

model2_9 = AutoencoderGRU_2(
    seq_length=seq_length,
    input_dim=1,
    embedding_dim=embedding_dim,
    teacher_forcing_ratio=teacher_forcing_ratio
)

trainer2_9 = get_trainer("ex2_9")
trainer2_9.fit(model2_9, train_B_loader_360, val_B_loader_360)


### 2-10

In [ ]:
seq_length = 432
embedding_dim = int(seq_length / 2)
teacher_forcing_ratio = 0

model2_10 = AutoencoderGRU_2(
    seq_length=seq_length,
    input_dim=1,
    embedding_dim=embedding_dim,
    teacher_forcing_ratio=teacher_forcing_ratio
)

trainer2_10 = get_trainer("ex2_10")
trainer2_10.fit(model2_10, train_B_loader_432, val_B_loader_432)


### dump loss

In [ ]:
experiment_2_loss = [model2_1.train_losses, model2_1.val_losses,
                     model2_2.train_losses, model2_2.val_losses,
                     model2_3.train_losses, model2_3.val_losses,
                     model2_4.train_losses, model2_4.val_losses,
                     model2_5.train_losses, model2_5.val_losses,
                     model2_6.train_losses, model2_6.val_losses,
                     model2_7.train_losses, model2_7.val_losses,
                     model2_8.train_losses, model2_8.val_losses,
                     model2_9.train_losses, model2_9.val_losses,
                     model2_10.train_losses, model2_10.val_losses]

with open('experiment_2_loss.pkl', 'wb') as f:
    pickle.dump(experiment_2_loss, f)

# 6. Result

## I. Dataset A

In [ ]:
print(np.array(model2_1.train_losses).mean())
print(np.array(model2_1.train_losses).std())

In [ ]:
test_A_loader_144 = get_loader(data=test_A, batch_size=32, sub_length=144, ratio=0.75, shuffle=False)
test_B_loader_144 = get_loader(data=test_B, batch_size=32, sub_length=144, ratio=0.75, shuffle=False)

In [ ]:
# Initialize the loss function, alpha is the same as during training  
loss_fn = CombinedLoss(alpha=0.5)  

# Modify the loss function to return the loss for each sample  
class CombinedLossPerSample(nn.Module):  
    def __init__(self, alpha=0.5):  
        super(CombinedLossPerSample, self).__init__()  
        self.alpha = alpha  
        self.mse_loss = nn.MSELoss(reduction='none')  

    def forward(self, y_true, y_pred):  
        # Compute MSE loss for each sample  
        mse = self.mse_loss(y_true, y_pred).mean(dim=[1, 2])  # Average over sequence length and feature dimensions  
        # Compute fluctuation-sensitive loss  
        delta_true = y_true[:, 1:] - y_true[:, :-1]  # Rate of change for true values  
        delta_pred = y_pred[:, 1:] - y_pred[:, :-1]  # Rate of change for predicted values  
        fluctuation_loss = ((delta_true - delta_pred) ** 2).mean(dim=[1, 2])  
        # Compute combined loss  
        combined_loss = self.alpha * mse + (1 - self.alpha) * fluctuation_loss  
        return combined_loss  # Return the loss for each sample  

# Use the new loss function  
loss_fn = CombinedLossPerSample(alpha=0.5)  

# %%  
# Collect predictions, true values, and losses for the training set  
train_preds = []  
train_targets = []  
train_losses = []  

model2_1.eval()  # Set model to evaluation mode  
with torch.no_grad():  
    for batch in train_A_loader_144:  
        x = batch[0].to(model2_1.device)  
        pred = model2_1(x)  
        loss = loss_fn(pred, x)  # Loss for each sample  
        train_preds.append(pred.cpu())  
        train_targets.append(x.cpu())  
        train_losses.extend(loss.cpu().numpy())  # Add loss for each sample to the list  

# %%  
# Collect predictions, true values, and losses for the validation set  
val_preds = []  
val_targets = []  
val_losses = []  

with torch.no_grad():  
    for batch in val_A_loader_144:  
        x = batch[0].to(model2_1.device)  
        pred = model2_1(x)  
        loss = loss_fn(pred, x)  # Loss for each sample  
        val_preds.append(pred.cpu())  
        val_targets.append(x.cpu())  
        val_losses.extend(loss.cpu().numpy())  

# %%  
# Calculate the average loss  
avg_train_loss = np.mean(train_losses)  
avg_val_loss = np.mean(val_losses)  

print(f'Average training loss: {avg_train_loss:.6f}')  
print(f'Average validation loss: {avg_val_loss:.6f}')  

# %%  
# Concatenate the predictions and true values across batches  
train_preds = torch.cat(train_preds, dim=0)  
train_targets = torch.cat(train_targets, dim=0)  

val_preds = torch.cat(val_preds, dim=0)  
val_targets = torch.cat(val_targets, dim=0)  

# %%  
# Compute errors  
train_errors = (train_preds - train_targets).numpy()  
val_errors = (val_preds - val_targets).numpy()  

# Flatten errors to a one-dimensional array  
train_errors = train_errors.flatten()  
val_errors = val_errors.flatten()  

# %%  
# Plot error distribution  
import matplotlib.pyplot as plt  
import seaborn as sns  

plt.figure(figsize=(10, 5))  
sns.histplot(train_errors, bins=100, kde=True, color='blue', label='Training Error')  
sns.histplot(val_errors, bins=100, kde=True, color='red', label='Validation Error')  
plt.legend()  
plt.title('Error Distribution')  
plt.xlabel('Error')  
plt.ylabel('Frequency')
plt.savefig('Error Distribution.svg', dpi=300) 
plt.show()  

# %%  
# Plot loss distribution  
plt.figure(figsize=(10, 5))  
sns.histplot(train_losses, bins=100, kde=True, color='blue', label='Training Loss')  
sns.histplot(val_losses, bins=100, kde=True, color='red', label='Validation Loss')  
plt.legend()  
plt.title('Loss Distribution')  
plt.xlabel('Loss')  
plt.ylabel('Frequency')  
plt.savefig('Loss Distribution.svg', dpi=300) 
plt.show()  

# %%  
# Calculate the mean and standard deviation of predictions for the training and validation sets  
train_mean = train_preds.mean().item()  
train_std = train_preds.std().item()  

val_mean = val_preds.mean().item()  
val_std = val_preds.std().item()  

print(f'Mean of training predictions: {train_mean:.4f}, Standard deviation: {train_std:.4f}')  
print(f'Mean of validation predictions: {val_mean:.4f}, Standard deviation: {val_std:.4f}')

In [ ]:

import numpy as np  
import matplotlib.pyplot as plt  
import torch  

# Define validation set mean and std dev of the loss  
mu_epsilon = 0.009993348002899438 
sigma_epsilon = 0.007432170408685299 

# Collect predictions, true values, and losses for the test set  
test_preds = []  
test_targets = []  

# Initialize the loss function, alpha is the same as during training  
loss_fn = CombinedLoss(alpha=0.5)  

model2_1.eval()  # Set model to evaluation mode  
with torch.no_grad():  
    for batch in test_A_loader_144:  
        x = batch[0].to(model2_1.device)  
        pred = model2_1(x)  
        test_preds.append(pred.cpu())  
        test_targets.append(x.cpu())  

# Concatenate predictions and true values across batches  
test_preds = torch.cat(test_preds, dim=0)  
test_targets = torch.cat(test_targets, dim=0)  

# Calculate per-time-point losses (using the same loss function as for training)  
errors = loss_fn(test_targets, test_preds).numpy()  

# Calculate anomaly probabilities for each time point  
anomaly_probabilities =  1 - np.exp(-0.5 * ((errors - mu_epsilon) / sigma_epsilon) ** 2) 

# Select the number of samples to plot  
num_examples = 5  # You can adjust this number as needed  

# Function to find contiguous anomaly regions
def find_anomaly_regions(anomaly_indices):
    if len(anomaly_indices) == 0:
        return []
    regions = []
    start = anomaly_indices[0]
    end = anomaly_indices[0]
    for idx in anomaly_indices[1:]:
        if idx == end + 1:
            end = idx
        else:
            regions.append((start, end))
            start = idx
            end = idx
    regions.append((start, end))
    return regions

def plot_result_A(i=1):
    plt.figure(figsize=(15, 10))  

    # Extract the i-th sample's input (true values), output (predicted values), and anomaly probabilities  
    input_seq = test_targets[i].numpy().flatten()  
    output_seq = test_preds[i].numpy().flatten()  
    anomaly_seq = anomaly_probabilities[i].flatten()  # Flatten the anomaly probabilities sequence  

    # Identify anomaly positions where anomaly probability > 0.8  
    anomaly_threshold = 0.8  
    anomaly_indices = np.where(anomaly_seq > anomaly_threshold)[0]  
    anomaly_regions = find_anomaly_regions(anomaly_indices)

    # Create a subplot structure with three rows and one column
    plt.subplot(3, 1, 1)  
    plt.plot(input_seq, label='Input Sequence', color='blue')  
    # Highlight anomaly regions on the input sequence  
    for start, end in anomaly_regions:
        plt.axvspan(start, end, color='red', alpha=0.3)
    plt.title(f'Test Sample {i+1} - Input Sequence')  
    plt.xlabel('Time Step')  
    plt.ylabel('Value')  
    plt.legend()  

    plt.subplot(3, 1, 2)  
    plt.plot(output_seq, label='Output Sequence', color='red')  
    plt.title(f'Test Sample {i+1} - Output Sequence')  
    plt.xlabel('Time Step')  
    plt.ylabel('Value')  
    plt.legend()  

    plt.subplot(3, 1, 3)  
    plt.plot(anomaly_seq, label='Anomaly Probability', color='green')  
    # Plot the anomaly probability threshold line  
    plt.axhline(y=anomaly_threshold, color='orange', linestyle='--', label='Threshold (0.8)')  
    # Highlight anomaly probabilities above the threshold  
    if len(anomaly_indices) > 0:  
        plt.scatter(anomaly_indices, anomaly_seq[anomaly_indices], color='red', label='Anomaly (>0.8)')  
    plt.title(f'Test Sample {i+1} - Anomaly Probability')  
    plt.xlabel('Time Step')  
    plt.ylabel('Probability')  
    plt.legend()  

    plt.tight_layout()  
    plt.savefig(f'Test Sample {i+1} - Anomaly Probability.svg', dpi=300)
    plt.show()

In [ ]:
plot_result_A(3)

In [ ]:
plot_result_A(4)

## II. Dataset B

In [ ]:
checkpoint_path_B = "checkpoint/ex2_1/epoch=92-val_loss=0.01.ckpt"
model_B = AutoencoderGRU_2.load_from_checkpoint(checkpoint_path_B)

In [ ]:
def plot_error(ts1, ts2, error_limit):
    error = (ts1 - ts2)**2

    fig, axs = plt.subplots(2, 1, figsize=(10, 8))

    axs[0].plot(ts1, label='Input')
    axs[0].plot(ts2, label='Output', linestyle='--')
    axs[0].set_title('Input and output comparison')
    
    over_tolerance = np.abs(error) > error_limit
    axs[0].scatter(np.where(over_tolerance), ts1[over_tolerance], color='red', label='Out of Tolerance (anormaly)')
    # axs[0].scatter(np.where(over_tolerance), ts2[over_tolerance], color='yellow', label='Out of Tolerance (TS2)')
    axs[0].legend(loc='upper right')

    axs[1].plot(error, color='red', label='Squared Error')
    axs[1].set_title('Reconstruction error')
    axs[1].axhline(y=error_limit, color='green', linestyle='--', label='Tolerance Threshold')
    axs[1].set_xlabel('Time Step')
    axs[1].set_ylabel('Squared Error')
    axs[1].legend(loc='upper right')

    plt.tight_layout()
    plt.show()

In [ ]:
test_A_loader_288 = get_loader(data=test_A, batch_size=32, sub_length=288, ratio=0.75, shuffle=False)
test_B_loader_288 = get_loader(data=test_B, batch_size=32, sub_length=288, ratio=0.75, shuffle=False)

In [ ]:
# seq_length=288
# embedding_dim = int(seq_length / 2) 
# teacher_forcing_ratio = 0

# checkpoint_path_1_1 = "checkpoint/ex1_1/epoch=89-val_loss=0.01.ckpt"
# model_1_1 = AutoencoderGRU_2.load_from_checkpoint(
#     checkpoint_path_1_1,
#     seq_length=seq_length,
#     input_dim=1,
#     embedding_dim=embedding_dim,
#     teacher_forcing_ratio=teacher_forcing_ratio
#     )

In [ ]:
trainer = pl.Trainer()

predictions = trainer.predict(model1_1, dataloaders=test_A_loader_288)

predicted_values = [pred.detach().cpu().numpy() for pred in predictions]

In [ ]:
def plot_in_out(batch, sample):
    for i, data in enumerate(test_A_loader_288):
        if i == batch:
            x = data[0]
            x = x[sample]
            break

    pred = predicted_values[batch][sample]

    print(x.shape)
    print(pred.shape)
    plot_error(x.numpy(), pred, 0.1)

In [ ]:
plot_in_out(2,7)

In [ ]:
model_2_1.train_losses

In [ ]:
test_A_loader_144 = get_loader(data=test_A, batch_size=32, sub_length=144, ratio=0.75, shuffle=False)
test_B_loader_144 = get_loader(data=test_B, batch_size=32, sub_length=144, ratio=0.75, shuffle=False)

In [ ]:
seq_length=144
embedding_dim = int(seq_length / 2)
teacher_forcing_ratio = 0 

checkpoint_path_2_1 = "checkpoint/ex2_1/epoch=93-val_loss=0.01.ckpt"
model_2_1 = AutoencoderGRU_2.load_from_checkpoint(
    checkpoint_path_2_1,
    seq_length=seq_length,
    input_dim=1,
    embedding_dim=embedding_dim,
    teacher_forcing_ratio=teacher_forcing_ratio,
    alpha=0.2
    )

In [ ]:
trainer = pl.Trainer()

predictions = trainer.predict(model2_1, dataloaders=test_A_loader_144)

predicted_values = [pred.detach().cpu().numpy() for pred in predictions]

In [ ]:
def plot_in_out(batch, sample):
    for i, data in enumerate(test_A_loader_144):
        if i == batch:
            x = data[0]
            x = x[sample]
            break

    pred = predicted_values[batch][sample]

    print(x.shape)
    print(pred.shape)
    plot_error(x.numpy(), pred, 0.05)

In [ ]:
plot_in_out(1, 18)